In [5]:
## virtual environment setup
%pip install conda
%pip install geopandas

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.7/341.7 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 2.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd

df2 = pd.read_csv('/Users/kaiweber/Documents/GitHub/DLBDSEDA02_Projekt_Advanced-Data-Analysis/src/complaints_data_cleaned.csv')
df2.head(20)


,author,posted_on,rating,text,state
0,"Alantae of Chesterfeild, MI","Nov. 22, 2016",1,I used to love Comcast. Until all these consta...,MI
1,"Vera of Philadelphia, PA","Nov. 19, 2016",1,I'm so over Comcast! The worst internet provid...,PA
2,"Sarah of Rancho Cordova, CA","Nov. 17, 2016",1,If I could give them a negative star or no sta...,CA
3,"Dennis of Manchester, NH","Nov. 16, 2016",1,I've had the worst experiences so far since in...,NH
4,"Ryan of Bellevue, WA","Nov. 14, 2016",1,Check your contract when you sign up for Comca...,WA
5,"Terri of Mobile, AL","Nov. 9, 2016",1,Thank God. I am changing to Dish. They gave me...,AL
6,"Kellie of Salt Lake City, UT","Nov. 9, 2016",1,I Have been a long time customer and only have...,UT
7,"Kathleen of New Haven, CT","Nov. 6, 2016",2,There is a malfunction on the DVR manager whic...,CT
8,"Shira of Bloomfield, NJ","Nov. 5, 2016",1,Charges overwhelming. Comcast service rep was ...,NJ
9,"Kristy of Alpharetta, GA","Nov. 2, 2016",1,"I have had cable, DISH, and U-verse, etc. in t...",GA


# Visualisierung der strukturierten Daten

In [41]:
import plotly.graph_objects as go
import pandas as pd

# df2 = pd.read_csv('/Users/kaiweber/Documents/GitHub/DLBDSEDA02_Projekt_Advanced-Data-Analysis/src/complaints_data_cleaned.csv')

print(f"Datensatz geladen: {len(df2)} Beschwerden")
print(f"Verfügbare Spalten: {df2.columns.tolist()}")

# Beschwerdeanzahlen pro Bundesstaat aggregieren
complaints_by_state = df2.groupby('state').size().reset_index(name='complaints')
complaints_by_state = complaints_by_state.sort_values('complaints', ascending=False)

# USA = 50 US-Bundesstaaten + DC
all_states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC'
]

all_states_df = pd.DataFrame({'state': all_states})                                                 # DataFrame mit allen Bundesstaaten erstellen
all_states_complaints = all_states_df.merge(complaints_by_state, on='state', how='left')            # Mit Beschwerdeanzahl mergen (fehlende Werte = 0)
all_states_complaints['complaints'] = all_states_complaints['complaints'].fillna(0).astype(int)

print(f"\nBeschwerdeanzahl pro Bundesstaat (inkl. fehlende):")
print(all_states_complaints.sort_values('complaints', ascending=False))

print(f"\n✓ Karte erstellen mit {len(all_states_complaints)} Bundesstaaten")
print(f"  - mit Beschwerden:  {(all_states_complaints['complaints'] > 0).sum()}")
print(f"  - ohne Beschwerden: {(all_states_complaints['complaints'] == 0).sum()}")


# Choroplethenkarte Choroplethenkarte (engl. choropleth map) mit allen Bundesstaaten
fig = go.Figure(data=go.Choropleth(
    locations=all_states_complaints['state'],                    # State-Codes
    z=all_states_complaints['complaints'].astype(float),         # Beschwerdeanzahl (0 für keine)
    text=all_states_complaints['state'],                         # State-Codes als Text für Hover
    locationmode='USA-states',
    colorscale=[
        [0, "#499615"],                                         # keine Beschwerden
        [0.5, "#fbde03"],                                       # durchschnittlich viele Beschwerden
        [1, '#8B0000']                                          # viele Beschwerden
    ],
    colorbar_title="Anzahl Beschwerden",
    hovertemplate='<b>%{text}</b><br>Beschwerden: %{z}<extra></extra>'
))

fig.update_layout(
    title_text='Beschwerden nach US-Bundesstaaten (Choroplethenkarte)',
    geo_scope='usa',
    height=600,
    width=1000
)

fig.show()

Datensatz geladen: 5627 Beschwerden
Verfügbare Spalten: ['author', 'posted_on', 'rating', 'text', 'state']

Beschwerdeanzahl pro Bundesstaat (inkl. fehlende):
   state  complaints
8     FL         775
4     CA         551
9     GA         413
12    IL         375
37    PA         323
29    NJ         274
42    TX         274
21    MI         265
41    TN         244
19    MD         237
45    VA         214
46    WA         214
20    MA         182
5     CO         180
13    IN         143
22    MN         108
36    OR         100
6     CT          75
43    UT          61
0     AL          55
28    NH          45
39    SC          41
50    DC          41
7     DE          40
30    NM          38
23    MS          35
31    NY          32
17    LA          28
2     AZ          22
16    KY          21
47    WV          19
3     AR          19
34    OH          12
24    MO          12
44    VT          12
48    WI           9
18    ME           7
32    NC           7
27    NV           5
1